<a href="https://colab.research.google.com/github/ianakoto/Cropland-Mapping/blob/main/GEO_AI_Challenge_for_Cropland_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEO-AI Challenge for Cropland Mapping

 **Project Description**:
  - Timely and accurate crop maps are essential for agriculture and research.
  - Current global cropland extent maps have limitations.
  - New high-resolution satellite data and machine learning offer solutions.
  - The project aims to advance global cropland mapping using remote sensing data.


  **Objectives**:
  1. Develop methods for annual cropland mapping at 10m resolution.
  2. Test the method's temporal extendibility at a local scale.


This project is broken into the following notebooks:

- **Open 🧭 Overview**: Go through what we want to achieve, and explore the data we want to use as inputs and outputs for our model.

- **Open 🗄️ Create the dataset**: Use Apache Beam to fetch data from Earth Engine in parallel, and create a dataset for our model in Dataflow.

- **Open 🧠 Train the model**: Build a Unet with pretained model and train it in Vertex AI with the dataset we created.

- **Open 🔮 Model predictions**: Get predictions from the model with data it has never seen before.

This sample leverages geospatial satellite data from Google Earth Engine. Using satellite imagery, you'll build and train a model for Cropland classification

## ☁️ My Google Cloud resources

Make sure you have followed these steps to configure your Google Cloud project:

1. Enable the APIs: _Earth Engine_

  <button>

  [Click here to enable the APIs](https://console.cloud.google.com/flows/enableapi?apiid=earthengine.googleapis.com)
  </button>

1. Register your
  [Compute Engine default service account](https://console.cloud.google.com/iam-admin/iam)
  on Earth Engine.

  <button>

  [Click here to register your service account on Earth Engine](https://signup.earthengine.google.com/#!/service_accounts)
  </button>

Once you have everything ready, you can go ahead and fill in your Google Cloud resources in the following code cell.
Make sure you run it!

In [1]:
from __future__ import annotations

import os
from google.colab import auth

# Please fill in these values.
project = "kagglex-396821"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"

# Authenticate to Colab.
auth.authenticate_user()

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

Updated property [core/project].


# 🧭 Overview

The goal of our model is using satellite images to do _cropland classification_. We want to advance global cropland mapping using remote sensing data, and machine learning. By using high resolution images like _sentinel 2_, we want to acheive state of the art cropland classfication that can be used generally on any region of interest.

Specifically, we want to classify the amount of rainfall, measured in millimeters per hour, for the next two to six hours in the future.

When working with satellite data, each image has the shape `(width, height, bands)`.
**Bands** contain _numeric values_ for each pixel in the image, like the measurements from specific satellite instruments for different ranges of the electromagnetic spectrum, or the probabilities of different classifications.
If you're familiar with image classification problems, you can think of the bands as similar to an image's RGB channels.

# 🛰️ Sentinel 2 Data in Cropland Mapping

In our project focused on advancing global cropland mapping using remote sensing data, we rely heavily on Sentinel 2 satellite imagery to achieve our objectives. Sentinel 2, operated by the European Space Agency (ESA), provides us with a valuable resource for obtaining high-resolution optical data that is crucial for accurate cropland mapping.

## 🌾 High-Resolution Imaging

One of the key advantages of Sentinel 2 data is its high spatial resolution. The satellite's multispectral sensors capture imagery at a ground resolution of 10 meters, allowing us to discern fine details on the Earth's surface. This level of detail is especially valuable when mapping cropland, as it enables us to differentiate between various crop types, observe field boundaries, and detect changes in land use with precision.

## 🌍 Global Coverage

Sentinel 2 provides global coverage, making it an ideal data source for our project's global cropland mapping objectives. We can access imagery from almost anywhere on Earth, ensuring that we can map cropland in diverse regions and countries.

## 📅 Temporal Extensibility

Our project's objectives include developing methods for annual cropland mapping and testing the temporal extendibility of these methods at a local scale. Sentinel 2 data excels in this regard due to its frequent revisit times. The satellite captures images of the same location every 5 days, providing a rich temporal dataset that allows us to monitor crop growth cycles, changes in land use, and seasonal variations in vegetation.

## 🌱 Vegetation Indices

To accurately identify cropland and assess its condition, we leverage Sentinel 2's spectral bands to calculate various vegetation indices, such as the Normalized Difference Vegetation Index (NDVI) and the Enhanced Vegetation Index (EVI). These indices help us monitor the health and vigor of crops throughout the growing season, aiding in the differentiation of cropland from other land cover types.

## 🧠 Machine Learning Integration

In conjunction with Sentinel 2 data, we employ machine learning techniques to analyze and classify the imagery. We train models to recognize patterns associated with cropland, enabling automated cropland mapping at scale. The high-quality and frequent Sentinel 2 data inputs are vital for training and validating these machine learning models.

In summary, Sentinel 2 data plays a pivotal role in our project's mission to advance global cropland mapping. Its high-resolution imaging, global coverage, frequent revisit times, and suitability for vegetation analysis make it an indispensable asset for our objectives. By harnessing the power of this satellite data alongside machine learning algorithms, we aim to provide timely and accurate cropland extent maps that are vital for agriculture and research worldwide.


## INSTALL DEPENDENCIES

In [2]:
!pip install -q earthengine-api
!pip install -q folium

## Import Earth Engine API and authenticate<a class="anchor" id="import-api"></a>

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [69]:
import ee
import folium
from folium import plugins
from IPython.display import Image
from datetime import datetime, timedelta
import io
import pandas as pd
import random
import numpy as np

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [4]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=9tUJVfZjyl8YCwtiqhJaEooxN0JumMI07NLUh-VnYHI&tc=p0TsAJeqjJzrOTteg1IDwHIm_4IgoY8Od_Flh-a_z7I&cc=BonvAoCy_PBPJBbfMKzyAYzQyIVaT6rh-Zr3ShdstXk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BW3s-XN93_YmJQ_NIGaNsA1c1mTz7J7XtIA6NLUNtdu5RCxt6ylUow

Successfully saved authorization token.


## Test API

In [5]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## Interactive SatelliteImage Display with Folium

In [10]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [8]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [30]:
def generate_color_palette(num_colors):
    """
    Generates a random color palette in hex format.

    Args:
        num_colors (int): The number of colors to generate.

    Returns:
        list: A list of hex color codes.
    """
    colors = []
    for i in range(num_colors):
        red = random.randint(0, 255)
        green = random.randint(0, 255)
        blue = random.randint(0, 255)
        hex_code = f"#{red:02x}{green:02x}{blue:02x}"
        colors.append(hex_code)
    return colors



## Explore Dataset

In [15]:
train_pd = pd.read_csv("/content/drive/MyDrive/Zindi Competitions/Cropland Classification/Train.csv")
sample_pd = pd.read_csv("/content/drive/MyDrive/Zindi Competitions/Cropland Classification/SampleSubmission.csv")
test_pd = pd.read_csv("/content/drive/MyDrive/Zindi Competitions/Cropland Classification/Test.csv")

In [16]:
train_pd.head()

,ID,Lat,Lon,Target
0,ID_SJ098E7S2SY9,34.162491,70.763668,0
1,ID_CWCD60FGJJYY,32.075695,48.492047,0
2,ID_R1XF70RMVGL3,14.542826,33.313483,1
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1
4,ID_C20R2C0AYIT0,14.419128,33.528450,0


In [17]:
sample_pd.head()

,ID,Target
0,ID_9ZLHTVF6NSU7,
1,ID_LNN7BFCVEZKA,
2,ID_SOYSG7W04UH3,
3,ID_EAP7EXXV8ZDE,
4,ID_QPRX1TUQVGHU,


In [18]:
test_pd.head()

,ID,Lat,Lon
0,ID_9ZLHTVF6NSU7,34.254835,70.348699
1,ID_LNN7BFCVEZKA,32.009669,48.535526
2,ID_SOYSG7W04UH3,14.431884,33.399991
3,ID_EAP7EXXV8ZDE,14.281866,33.441224
4,ID_QPRX1TUQVGHU,14.399365,33.109566


### Load Countries Dataset and get **ROI** of the following:
- Iran (Islamic Republic of)
- Sudan
- Afghanistan

In [23]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

# Filter countries by name to obtain the ROIs for Iran, Sudan, and Afghanistan
iran_roi = countries.filter(ee.Filter.eq('country_na', 'Iran'))
sudan_roi = countries.filter(ee.Filter.eq('country_na', 'Sudan'))
afghanistan_roi = countries.filter(ee.Filter.eq('country_na', 'Afghanistan'))

# Get the geometries of the ROIs
iran_geometry = iran_roi.geometry()
sudan_geometry = sudan_roi.geometry()
afghanistan_geometry = afghanistan_roi.geometry()


Display the ROIs

In [26]:
# Create a folium map object.
roi_map = folium.Map(location=[20, 0], zoom_start=3, height=500)
roi_map.add_ee_layer(iran_geometry, {}, 'ROI of Iran')
roi_map.add_ee_layer(sudan_geometry, {}, 'ROI of Sudan')
roi_map.add_ee_layer(afghanistan_geometry, {}, 'ROI of Afghanistan')

# Add a layer control panel to the map.
roi_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(roi_map)

# Display the map.
display(roi_map)

## Now Explore Sentinel 2

In [68]:
LABEL = "is_crop_or_land"
IMAGE_COLLECTION = "COPERNICUS/S2"
BANDS = [
    "B1",
    "B2",
    "B3",
    "B4",
    "B5",
    "B6",
    "B7",
    "B8",
    "B8A",
    "B9",
    "B10",
    "B11",
    "B12",
]
FEATURES = ["NDVI", "EVI"]
SCALE = 10
PATCH_SIZE = 16

In [43]:
def calculate_ndvi(image):
    """Calculate NDVI for an image."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def calculate_evi(image):
    """Calculate EVI for an image."""
    evi = image.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))', {
            'B8': image.select('B8'),
            'B4': image.select('B4'),
            'B2': image.select('B2')
        }
    ).rename('EVI')
    return image.addBands(evi)

In [49]:
def create_composited_sentinel2_collection(roi,
                                           start_date_str,
                                           end_date_str,
                                           interval=15,
                                           limit=10,
                                           include_ndvi=True,
                                           include_evi=True):
    """
    Creates a 15-day composited Sentinel-2 image collection within the specified ROI and time range.

    Args:
        roi (ee.Geometry): The region of interest as an Earth Engine geometry.
        start_date_str (str): The start date in 'yyyy-mm-dd' format.
        end_date_str (str): The end date in 'yyyy-mm-dd' format.
        interval (int, optional): The number of days for each composite interval. Default is 15.
        limit (int, optional): The maximum number of images to include in each composite. Default is 10.
        include_ndvi (bool, optional): Whether to calculate and include NDVI bands. Default is True.
        include_evi (bool, optional): Whether to calculate and include EVI bands. Default is True.

    Returns:
        ee.ImageCollection: The composited Sentinel-2 image collection.
    """
    # Convert start_date_str and end_date_str to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Create an empty ImageCollection to store the composites
    composited_collection = ee.ImageCollection([])

    # Loop through the dates and create composites
    while start_date < end_date:
        # Calculate the end date for the current interval
        interval_end_date = start_date + timedelta(days=interval)
        if interval_end_date > end_date:
            interval_end_date = end_date

        # Format dates as strings
        interval_start_date_str = start_date.strftime('%Y-%m-%d')
        interval_end_date_str = interval_end_date.strftime('%Y-%m-%d')

        # Filter Sentinel-2 data for the current date range
        collection = ee.ImageCollection('COPERNICUS/S2') \
            .filterBounds(roi) \
            .filterDate(interval_start_date_str, interval_end_date_str) \
            .select(BANDS) \
            .sort('CLOUDY_PIXEL_PERCENTAGE') \
            #.limit(limit)

        # Calculate NDVI and EVI if requested
        if include_ndvi:
            collection = collection.map(calculate_ndvi)
        if include_evi:
            collection = collection.map(calculate_evi)

        # Create a median composite of the image collection
        median_image = collection.median()

        # Add the composite to the ImageCollection
        composited_collection = composited_collection.merge(ee.ImageCollection([median_image]))

        # Move to the next interval
        start_date = interval_end_date + timedelta(days=1)

    return composited_collection


In [72]:
def select_collection_by_point(point,
                               iran_geometry,
                               sudan_geometry,
                               afghanistan_geometry,
                               iran_collection,
                               sudan_collection,
                               afghanistan_collection):
    """
    Selects an image collection based on whether a given point is within the bounds of a geometry.

    Args:
        point (ee.Geometry.Point): The point to check.
        iran_geometry (ee.Geometry): The geometry representing the bounds of Iran.
        sudan_geometry (ee.Geometry): The geometry representing the bounds of Sudan.
        afghanistan_geometry (ee.Geometry): The geometry representing the bounds of Afghanistan.
        iran_collection (ee.ImageCollection): The Sentinel-2 image collection for Iran.
        sudan_collection (ee.ImageCollection): The Sentinel-2 image collection for Sudan.
        afghanistan_collection (ee.ImageCollection): The Sentinel-2 image collection for Afghanistan.

    Returns:
        ee.ImageCollection or None: The selected image collection or None if the point is not within any geometry.
    """
    # Check if the point is within the bounds of the geometries
    is_in_iran = iran_geometry.contains(point)
    is_in_sudan = sudan_geometry.contains(point)
    is_in_afghanistan = afghanistan_geometry.contains(point)

    # Depending on the results, choose which collection to return
    if is_in_iran.getInfo():
        selected_collection = iran_collection
    elif is_in_sudan.getInfo():
        selected_collection = sudan_collection
    elif is_in_afghanistan.getInfo():
        selected_collection = afghanistan_collection
    else:
        selected_collection = None  # Point is not within any of the geometries

    return selected_collection


In [58]:
start_date = '2022-1-1'
end_date = '2022-12-31'

iran_collection = create_composited_sentinel2_collection(iran_geometry, start_date, end_date)
sudan_collection = create_composited_sentinel2_collection(sudan_geometry, start_date, end_date)
afghanistan_collection = create_composited_sentinel2_collection(afghanistan_geometry, start_date, end_date)



In [62]:
# Get the collection by point
latitude = 32.4279
longitude = 53.6880
point = ee.Geometry.Point([longitude, latitude])

# Call the function to select the appropriate collection
selected_collection = select_collection_by_point(
    point,
    iran_geometry,
    sudan_geometry,
    afghanistan_geometry,
    iran_collection,
    sudan_collection,
    afghanistan_collection
)


iran


In [63]:
print(selected_collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1,

Display Iran SENTINEL DATA

In [52]:
band_viz = {
    'bands': ['B4', 'B3', 'B2'],  # Replace with desired bands (e.g., B4=Red, B3=Green, B2=Blue)
    'min': 0,
    'max': 3000,  # Adjust the min and max values as needed
    'gamma': 1.4
}

ndviParams = {'min': -1,
              'max': 1,
              'palette': ['blue', 'white', 'green']
              }


# Create a folium map object.
sentinel_map = folium.Map(location=[32.4279, 53.6880],
                          zoom_start=5, height=600)

sentinel_map.add_ee_layer(selected_collection.mean().clip(iran_geometry),
                          band_viz,
                          'Sentinel Map of Iran')

sentinel_map.add_ee_layer(selected_collection.select("NDVI").mean().clip(iran_geometry),
                          ndviParams,
                          'NDVI Map of Iran')

# Add a layer control panel to the map.
sentinel_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(sentinel_map)

# Display the map.
display(sentinel_map)

## Merge 🏷️ labels + 🛰️ Sentinel image data

In the cell of the training dataframe, we write a function to extract the Sentinel image taken at the specific latitude/longitude for each row of our dataframe.

Then, using the **neighorboodToArray** method we create a FeatureCollection that contains the satellite data for each band at the latitude and longitude of interest as well as a 16 pixel padding around that point.

In [54]:
train_pd.head()

,ID,Lat,Lon,Target
0,ID_SJ098E7S2SY9,34.162491,70.763668,0
1,ID_CWCD60FGJJYY,32.075695,48.492047,0
2,ID_R1XF70RMVGL3,14.542826,33.313483,1
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1
4,ID_C20R2C0AYIT0,14.419128,33.528450,0


In [73]:
def labeled_feature(row):
    selected_bands = [BANDS] + [FEATURES] + [LABEL]

    select_point = ee.Geometry.Point([row.Lon, row.Lat])

    selected_collection = select_collection_by_point(
        select_point,
        iran_geometry,
        sudan_geometry,
        afghanistan_geometry,
        iran_collection,
        sudan_collection,
        afghanistan_collection
    )

    image = (
        selected_collection
        .select(BANDS)
        .mosaic()
    )
    point = ee.Feature(
        select_point,
        {LABEL: row.Target},
    )
    return (
        image.neighborhoodToArray(ee.Kernel.square(PATCH_SIZE))
        .sampleRegions(ee.FeatureCollection([point]), scale=SCALE)
        .first()
    )

In [74]:
df_subset = train_pd.head(100)
train_features = [labeled_feature(row) for row in df_subset.itertuples()]

To get a better sense of what's going on, let's look at the properties for the first Feature in the train_features list. You can see that it contains a property for the label **is_crop_or_land**, and 15 additional properies, one for each spectral band.

In [77]:
ee.FeatureCollection(train_features[50]).propertyNames().getInfo()

['system:index',
 'is_crop_or_land',
 'B10',
 'B11',
 'B12',
 'B8A',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9']

The data contained in each band property is an array of shape 33x33.

For example, here is the data for band B1 in the first element in our list expressed as a numpy array.

In [76]:
example_feature = np.array(train_features[0].get("B1").getInfo())
print(example_feature)
print("shape: " + str(example_feature.shape))

[[2900 2900 2900 ... 2851 2851 2839]
 [2900 2900 2900 ... 2851 2851 2839]
 [2885 2885 2885 ... 2876 2876 2835]
 ...
 [2858 2858 2858 ... 2858 2858 2858]
 [2858 2858 2858 ... 2858 2858 2858]
 [2858 2858 2858 ... 2862 2862 2862]]
shape: (33, 33)
